# TP - Mesurez la puissance des forêts aléatoires

Le fonctionnement des forêts aléatoires doit maintenant être un peu plus clair pour vous. Il est temps de passer à la pratique en observant les performances de ce type de modèle sur des données réelles 🙂

Dans ce chapitre, on va appliquer l’algorithme des forêts aléatoires sur un exemple concret. Le jeu de données que j’ai choisi est assez connu : il permet de reconnaître l’activité physique à partir de données du smartphone. Il est simple mais possède de nombreuses variables (> 500) ce qui va nous permettre d’étudier un certain nombre de choses. Prêt·e ? Affutez votre Notebook et téléchargez le nouveau dataset ici.

Le dataset
En plus de charger le dataset, vous pouvez aussi observer le fichier de description des différentes variables afin d’avoir une meilleur idée des données à disposition.

La connaissance du domaine est très importante pour un data scientist. Plus vous serez renseigné vos données, plus vous saurez créer des modèles efficaces rapidement. Vous ne devriez pas travailler à l’aveugle sur des variables muettes. C’est aussi important en amont pour avoir des hypothèses à confirmer par analyse exploratoire.

Dans un premier temps, étudions le dataset à notre disposition : le "Human Activity Recognition Using Smartphones Data Set".

Ce jeu de données contient les logs de capteurs de smartphone sur une trentaine d'individus en train d'effectuer des activités (s'assoir, se mettre debout, marcher, etc). L'objectif sera de prédire à partir des logs de capteurs le type d'activités que le sujet est en train d'effectuer.

En regardant le fichier de description du dataset, on peut observer qu'il y a beaucoup de features (561). D'emblée, en observant ce qu'elles désignent, on peut se dire qu'il y a une certaine redondance entre toutes ces variables. Dans un premier temps, on va effectuer une modélisation "brute" sans se soucier de nettoyer le jeu de données.

Dans un second temps, on va utiliser cette première modélisation pour mieux comprendre le dataset et ainsi effectuer une seconde modélisation plus efficace en éliminant des variables peu importantes.

On commence par charger les données.

In [4]:
# importing modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [5]:
# read data
train_data = pd.read_csv("C:/Users/Khadim Rassoul SENE/Documents/MY Datas/Human_Activity_Data/train.csv")
test_data = pd.read_csv("C:/Users/Khadim Rassoul SENE/Documents/MY Datas/Human_Activity_Data/test.csv")

In [6]:
train_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [7]:
train_data.shape

(7352, 563)

In [8]:
train_data.describe()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject
count,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,...,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000
mean,0.274488,-0.017695,-0.109141,-0.605438,-0.510938,-0.604754,-0.630512,-0.526907,-0.606150,-0.468604,...,-0.307009,-0.625294,0.008684,0.002186,0.008726,-0.005981,-0.489547,0.058593,-0.056515,17.413085
std,0.070261,0.040811,0.056635,0.448734,0.502645,0.418687,0.424073,0.485942,0.414122,0.544547,...,0.321011,0.307584,0.336787,0.448306,0.608303,0.477975,0.511807,0.297480,0.279122,8.975143
min,-1.000000,-1.000000,-1.000000,-1.000000,-0.999873,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-0.995357,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
25%,0.262975,-0.024863,-0.120993,-0.992754,-0.978129,-0.980233,-0.993591,-0.978162,-0.980251,-0.936219,...,-0.542602,-0.845573,-0.121527,-0.289549,-0.482273,-0.376341,-0.812065,-0.017885,-0.143414,8.000000
50%,0.277193,-0.017219,-0.108676,-0.946196,-0.851897,-0.859365,-0.950709,-0.857328,-0.857143,-0.881637,...,-0.343685,-0.711692,0.009509,0.008943,0.008735,-0.000368,-0.709417,0.182071,0.003181,19.000000
75%,0.288461,-0.010783,-0.097794,-0.242813,-0.034231,-0.262415,-0.292680,-0.066701,-0.265671,-0.017129,...,-0.126979,-0.503878,0.150865,0.292861,0.506187,0.359368,-0.509079,0.248353,0.107659,26.000000
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,1.000000,0.478157,1.000000,30.000000


In [9]:
test_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978,2,STANDING
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898,2,STANDING
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346,2,STANDING
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108,2,STANDING
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857,2,STANDING


In [10]:
test_data.shape

(2947, 563)

In [11]:
test_data.describe()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject
count,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,...,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000
mean,0.273996,-0.017863,-0.108386,-0.613635,-0.508330,-0.633797,-0.641278,-0.522676,-0.637038,-0.462063,...,-0.277593,-0.598756,0.005264,0.003799,0.040029,-0.017298,-0.513923,0.074886,-0.048720,12.986427
std,0.060570,0.025745,0.042747,0.412597,0.494269,0.362699,0.385199,0.479899,0.357753,0.523916,...,0.317245,0.311042,0.336147,0.445077,0.634989,0.501311,0.509205,0.324300,0.241467,6.950984
min,-0.592004,-0.362884,-0.576184,-0.999606,-1.000000,-0.998955,-0.999417,-0.999914,-0.998899,-0.952357,...,-1.000000,-1.000000,-1.000000,-0.993402,-0.998898,-0.991096,-0.984195,-0.913704,-0.949228,2.000000
25%,0.262075,-0.024961,-0.121162,-0.990914,-0.973664,-0.976122,-0.992333,-0.974131,-0.975352,-0.934447,...,-0.517494,-0.829593,-0.130541,-0.282600,-0.518924,-0.428375,-0.829722,0.022140,-0.098485,9.000000
50%,0.277113,-0.016967,-0.108458,-0.931214,-0.790972,-0.827534,-0.937664,-0.799907,-0.817005,-0.852659,...,-0.311023,-0.683672,0.005188,0.006767,0.047113,-0.026726,-0.729648,0.181563,-0.010671,12.000000
75%,0.288097,-0.010143,-0.097123,-0.267395,-0.105919,-0.311432,-0.321719,-0.133488,-0.322771,-0.009965,...,-0.083559,-0.458332,0.146200,0.288113,0.622151,0.394387,-0.545939,0.260252,0.092373,18.000000
max,0.671887,0.246106,0.494114,0.465299,1.000000,0.489703,0.439657,1.000000,0.427958,0.786436,...,1.000000,1.000000,0.998898,0.986347,1.000000,1.000000,0.833180,1.000000,0.973113,24.000000


In [12]:
X_train = train_data[train_data.columns[:-2]]
y_train = train_data['Activity']

X_test = test_data[test_data.columns[:-2]]
y_test = test_data['Activity']

Les données représentent des vecteurs de différentes mesures effectuées par le téléphone (accélération, secousses, etc). La dernière colonne représente l'activité, c'est ce qu'on va essayer de prédire à partir du reste. On va d'abord éliminer les features redondantes (intuitivement, les coordonnées polaires et cartésiennes doivent être corrélées par exemple ... ) Une première manière de faire serait de réfléchir et se renseigner sur le domaine d'études en question pour pouvoir éliminer des variables qui transmettent des informations similaires où n'influencent pas ou très peu la prédiction que l'on veut effectuer.

La seconde manière est d'utiliser justement une forêt aléatoire (!) de laquelle on va extraire l'importance des features qui la constituent, et ainsi déterminer quelles sont les features les plus importantes à partir de ça.

Une fois les données chargées, on peut déclarer un nouveau modèle de forêts aléatoires pour la classification logiquement appelé dans scikit   sklearn.RandomForestClassifier . On définit comme hyperparamètres 500 pour le nombre d'arbres. 

In [13]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=500, oob_score=True)

On peut maintenant entraîner notre modèle sur les données brutes, sans autre forme de procès.

In [14]:
model = rfc.fit(X_train, y_train)

In [15]:
# Voyons les  performances de ce modèle sur le jeu de données d'entraînement

from sklearn.metrics import accuracy_score

pred = rfc.predict(X_test)
print("accuracy {:.2f}".format(accuracy_score(y_test, pred)))

accuracy 0.93


Pas mal 😎  !

Maintenant que notre modèle est créé, on peut effectuer une sélection des features les plus importantes. Pour cela on va utiliser la fonction  SelectFromModel  qui utilise la propriété du modèle qu'on vient de créer  model.feature_importances_  qui permet d'évaluer l'importance relative des features fournies à la base (sur une échelle de 0 à 1). Intuitivement, cette importance est calculée en considérant que plus une feature est haute, plus elle contribue à une fraction plus élevée du jeu de donnée d'entraînement et donc des données au global. On considère donc qu'elle a plus d'importance que les features plus bas dans l'arbre. Cette fraction est utilisée comme estimateur de l'importance de la feature dans cet arbre, qu'on peut ensuite généraliser à la forêt entière. 

Si on a peu de features, on pourrait les afficher sur un histogramme afin d'évaluer à l'œil si il n'y a pas déjà une sélection à faire comme ici.

Donc en utilisant  SelectFromModel  avec un seuil d'importance choisi à l'aide de l'argument  threshold  , on peut créer une sélection des features qui sont les plus importantes à la création d'un modèle.

In [16]:
from sklearn.feature_selection import SelectFromModel
select = SelectFromModel(rfc, prefit=True, threshold=0.003)
X_train2 = select.transform(X_train)
print(X_train2.shape)

(7352, 86)


On a divisé par 5 le nombre de features utilisées, pas mal mais voyons si les performances restent similaires. À l'aide de l'argument threshold, on peut choisir le seuil d'importance que l'on souhaite pour les features à sélectionner.  On calcule en même temps le gain de temps car c'est ce qui nous intéresse principalement dans l'amélioration des performances.

In [17]:
import timeit

rfc2 = RandomForestClassifier(n_estimators=500, oob_score=True)

start_time = timeit.default_timer()

rfc2 = rfc2.fit(X_train2, y_train)

X_test2 = select.transform(X_test)

pred = rfc2.predict(X_test2)
elapsed = timeit.default_timer() - start_time
accuracy = accuracy_score(y_test, pred)

print("accuracy {:.2f} time {:.2f}s".format(accuracy, elapsed))

accuracy 0.89 time 56.73s


Et pour le modèle avec toutes les features

accuracy 0.93 time 50.04s

On a donc plus que divisé par trois le temps de calcul, sans trop perdre en performances ! C'est plus que respectable pour un premier jet.

# Conclusion

A vous maintenant de bidouiller afin d'améliorer les performances du modèle finale en modifiant les différents hyperparamètres de contrôle et trouver la bonne balance entre performances de classification / temps de calcul. Bien sûr le ratio souhaité dépendra de la problématique : est-ce qu'on veut une très bonne performance et peu importe le temps de calcul, ou bien un temps de calcul le plus rapide possible mais avec une performance minimum etc..